**SRP 04/14/2023:**

**PURPOSE:** Extract all of the gnats data from the database. Extract the underway data separately from the discrete and bottle data.

*Make sure the mysql_env is active.*

In [28]:
import mysql.connector
import pandas as pd
import numpy as np

In [29]:
# Connect to the database:

conn = mysql.connector.connect(host='ocean-optics-do-user-914951-0.b.db.ondigitalocean.com', user='balchdb', password='AVNS_H8z7U1SW5YTqL2XdIwZ', port=25060, database='globalDB')
cursor = conn.cursor()

#### Underway GNATS Extraction:

Include 4 clauses:
* SAS Query
* Ac9 Query
* GNATS Query
* UW Data Table

In [30]:
# Combine All Underway data: Underway, Ac9, and SAS.
# DO NOT include nav + basics, which pulls from the station info table. In the station info table, Bruce had matched UW IDs to stations that are upwards of 6 hours away from the uw data points. 
# Note that the SAS data table contains HyperSAS data in the correct reduced set of channels.
# Specify to only pull the SAS data columns and the Ac9 Data columns (not the UW id, or SAS or Ac9 ids). If this is not specified, then we end up with three UWID columns that are all labelled
# UWid and are thus very hard to deal with in Pandas.


### Specify SASDataTable Columns for query: ###

sasWvs = [412, 441, 490, 510, 533, 555, 671, 684, 780, 866]
sasSensor = ['li', 'lt', 'es']
sasQuery = ''

for sensor in sasSensor:
    for wv in sasWvs:
        sasQuery += 'SASDataTable.' + sensor + str(wv) + ', '
sasQuery += 'SASDataTable.ZenithAngle, SASDataTable.AzimuthAngle'

### Specify ac9DataTable Columns for query: ###

ac9Wvs = [412, 440, 488, 510, 550, 630, 650, 676, 715]
ac9Channel = ['agp','ag','cgp','cg']
ac9Query = ''

for channel in ac9Channel:
    for wv in ac9Wvs:
        ac9Query += 'ac9DataTable.' + channel + str(wv) + ', '
ac9Query = ac9Query[0:-2] #eliminate the space and column at the end

### Specify to select GNATS cruises only based on CruiseNames starting with 's':

gnatsQuery = "WHERE CruiseName LIKE 's%'"


### Construct and execute Query, and save records into a pandas dataframe: ###
cmd1 = "SELECT CruiseTable.CruiseName, UWDataTable.*, " + sasQuery + ', ' + ac9Query + " FROM CruiseTable "
cmd2 = "RIGHT JOIN UWDataTable ON CruiseTable.CruiseID = UWDataTable.CruiseID "
cmd3 = "LEFT JOIN SASDataTable ON UWDataTable.UWid = SASDataTable.UWid "
cmd4 = "Left JOIN ac9DataTable ON UWDataTable.UWid = ac9DataTable.UWid "
cmd5 = gnatsQuery + ";"
sql_uw =  cmd1 + cmd2 + cmd3 + cmd4 + cmd5
cursor.execute(sql_uw)
uw = cursor.fetchall()
uw_df = pd.DataFrame(uw, columns=[i[0] for i in cursor.description])

In [31]:
uw_df

,CruiseName,UWid,CruiseID,UWTime,UWLongitude,UWLatitude,UWStation,Cast,Temperature,Salinity,...,cgp715,cg412,cg440,cg488,cg510,cg550,cg630,cg650,cg676,cg715
0,s980917w,103883,21,1998-09-17 13:46:48,-66.3078,43.7736,None,NaN,9.98,31.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,s980917w,103884,21,1998-09-17 13:47:13,-66.3102,43.7732,None,NaN,10.00,31.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,s980917w,103885,21,1998-09-17 13:47:35,-66.3124,43.7729,None,NaN,10.04,31.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,s980917w,103886,21,1998-09-17 13:47:56,-66.3147,43.7724,None,NaN,10.04,31.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,s980917w,103887,21,1998-09-17 13:48:18,-66.3173,43.7721,None,NaN,10.00,31.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57741,s200901w,190533,296,2020-09-02 02:03:03,-69.7073,43.5703,None,NaN,17.35,31.50,...,0.929569,0.536559,0.424595,0.317085,0.287339,0.238598,0.186099,0.176491,0.168375,0.150433
57742,s200901w,190534,296,2020-09-02 02:07:54,-69.7251,43.5693,None,NaN,17.42,31.46,...,0.890651,0.541746,0.428639,0.319381,0.289362,0.240275,0.187302,0.177614,0.169454,0.151287
57743,s200901w,190535,296,2020-09-02 02:12:45,-69.7428,43.5682,None,NaN,17.52,31.46,...,1.022917,0.523348,0.412656,0.306193,0.277261,0.229847,0.179065,0.169926,0.162108,0.145053
57744,s200901w,190536,296,2020-09-02 02:17:29,-69.7601,43.5671,2020014,NaN,17.67,31.46,...,1.112557,0.674250,0.563397,0.452474,0.424135,0.371711,0.314120,0.303766,0.294704,0.274854


#### Discrete GNATS Extraction:

Include:
* Nav + Basics (as described in the G2 Reader)
    * StationInfoTable: StationNumber, StationTime, Latitude, Longitude
    * CruiseTable.CruiseName
* StationDataTable
* BatesDataTable
* AikenDataTable
* FlowcamDataTable
* Only GNATS cruises

In [32]:
### NAV + BASICS ###
basicsQuery = "CruiseTable.CruiseName, StationInfoTable.StationNumber, StationInfoTable.StationTime, StationInfoTable.Longitude, StationInfoTable.Latitude, StationDataTable.*"

### BATES ###
# Get a list of non-id columns in the batesDataTable:
cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='BatesDataTable'")
batesCols = cursor.fetchall()
batesCols = [col[0] for col in batesCols if 'ID' not in col[0]]

# Construct the bates Data Table query:
batesQuery = ''

for col in batesCols:
    batesQuery += 'BatesDataTable.' + col + ', '
batesQuery = batesQuery[0:-2]


### AIKEN ###
# Get a list of non-id columns in the Aiken Data Table:
cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='AikenDataTable'")
aikenCols = cursor.fetchall()
aikenCols = [col[0] for col in aikenCols if 'ID' not in col[0]]

# Construct the bates Data Table query:
# Note that one of the aiken columns has a hyphen in the name (pH-aiken). Therefore, we need to encapsulate the column name in quotes.
# This is why the aikenQuery is structured differently from the bates and flowcam queries.
aikenQuery = ''

for col in aikenCols:
    #aikenQuery += 'AikenDataTable.' + col + ', '
    aikenQuery += "'" + col + "', "
aikenQuery = aikenQuery[0:-2]


### FLOWCAM ###
# Get a list of non-id columns in the Aiken Data Table:
cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='FlowcamDataTable'")
fcamCols = cursor.fetchall()
fcamCols = [col[0] for col in fcamCols if 'ID' not in col[0]]

# Construct the bates Data Table query:
fcamQuery = ''

for col in fcamCols:
    fcamQuery += 'FlowcamDataTable.' + col + ', '
fcamQuery = fcamQuery[0:-2]


### GNATS ###
### Specify to select GNATS cruises only based on CruiseNames starting with 's':

gnatsQuery = "WHERE CruiseName LIKE 's%'"

In [33]:
#sql_discrete = "SELECT " + basicsQuery + ", " batesQuery + ", " aikenQuery + ", " fcamQuery + " FROM CruiseTable  " + gnatsQuery + ";"
cmd1 = "SELECT " + basicsQuery + ", " + batesQuery + ", " + aikenQuery + ", " + fcamQuery + " FROM CruiseTable "
#cmd1 = "SELECT * FROM CruiseTable "
cmd2 = "RIGHT JOIN StationInfoTable ON CruiseTable.CruiseID = StationInfoTable.CruiseID "
cmd3 = "RIGHT JOIN StationDataTable ON StationInfoTable.StationInfoID = StationDataTable.StationInfoID "
cmd4 = "LEFT JOIN BatesDataTable ON StationDataTable.StationDataID = BatesDataTable.StationDataID "
cmd5 = "LEFT JOIN AikenDataTable ON StationDataTable.StationDataID = AikenDataTable.StationDataID "
cmd6 = "LEFT JOIN FlowcamDataTable on StationDataTable.StationDataID = FlowcamDataTable.StationDataID "

sql_discrete = cmd1 + cmd2 + cmd3 + cmd4 + cmd5 + cmd6 + gnatsQuery
cursor.execute(sql_discrete)
discrete = cursor.fetchall()
discrete_df = pd.DataFrame(discrete, columns=[i[0] for i in cursor.description])  

### GNATS XBT Extraction:

The XBT data is found in both the StationInfoTable and in the ProfileDataTable.

StationInfoTable XBT Data:
* Latitude
* Longitude
* StationTime
* MixedLayerDepth
* Gradient
* WHERE CastType = 'XBT'

ProfileDataTable XBT Data:
* Depth
* Temperature1


In [34]:
xbtQuery = "SELECT StationInfoTable.StationInfoID, CastType, StationTime, Latitude, Longitude, Depth, Temperature1 " + \
            "FROM CruiseTable RIGHT JOIN StationInfoTable ON CruiseTable.CruiseID = StationInfoTable.CruiseID " + \
            "RIGHT JOIN ProfileDataTable ON StationInfoTable.StationInfoID = ProfileDataTable.StationInfoID " + \
            "WHERE CastType='XBT'" + \
            "AND CruiseName LIKE 's%'"

cursor.execute(xbtQuery)
xbt = cursor.fetchall()
xbt_df = pd.DataFrame(xbtData, columns=[i[0] for i in cursor.description])

In [35]:
xbt_df

,StationInfoID,CastType,StationTime,Latitude,Longitude,Depth,Temperature1
0,27203,XBT,2015-09-01 13:05:56,43.770500,-66.284400,0.00,12.99
1,27203,XBT,2015-09-01 13:05:56,43.770500,-66.284400,0.63,12.68
2,27203,XBT,2015-09-01 13:05:56,43.770500,-66.284400,1.26,12.60
3,27203,XBT,2015-09-01 13:05:56,43.770500,-66.284400,1.89,12.60
4,27203,XBT,2015-09-01 13:05:56,43.770500,-66.284400,2.52,12.59
...,...,...,...,...,...,...,...
252533,34635,XBT,2020-09-02 02:19:45,43.566528,-69.767668,113.76,7.25
252534,34635,XBT,2020-09-02 02:19:45,43.566528,-69.767668,114.42,7.24
252535,34635,XBT,2020-09-02 02:19:45,43.566528,-69.767668,115.08,7.25
252536,34635,XBT,2020-09-02 02:19:45,43.566528,-69.767668,115.74,7.26


#### Check Data:

Check that the resultant underway and discrete dataframes possess the expected dimensions.

In [36]:
### UNDERWAY ###
# Check if there are any duplicate columns by values:
print('No UW Duplicate columns by Values:', all(uw_df.columns.duplicated()==False))

# Check if there are any duplicate columns by name:
uw_cols = [col for col in uw_df.columns]
uw_cols_count = [uw_cols.count(col) for col in uw_cols]
print('No UW Duplicate columns by Name:', all([count==1 for count in uw_cols_count]))

# Check Number of Rows:
sql_uwData = "SELECT CruiseTable.CruiseName, UWDataTable.* FROM CruiseTable RIGHT JOIN UWDataTable ON CruiseTable.CruiseID = UWDataTable.CruiseID WHERE CruiseTable.CruiseName LIKE 's%';"
cursor.execute(sql_uwData)
uwData = cursor.fetchall()
uwData_df = pd.DataFrame(uwData, columns=[i[0] for i in cursor.description])
print("Underway Dataframe # rows matches # rows in UWDataTable:", len(uw_df)==len(uwData_df))


### DISCRETE ###
# Check if there are any duplicate columns by values:

print('No Discrete Duplicate columns by Values:', all(discrete_df.columns.duplicated()==False))

# Check if there are any duplicate columns by name:
d_cols = [col for col in discrete_df.columns]
d_cols_count = [d_cols.count(col) for col in d_cols]
print('No Discrete Duplicate columns by Name:', all([count==1 for count in d_cols_count]))

# Check Number of Rows: 
sql_stationData = "SELECT * FROM CruiseTable RIGHT JOIN StationInfoTable ON CruiseTable.CruiseID = StationInfoTable.CruiseID RIGHT JOIN StationDataTable ON StationInfoTable.StationInfoID = StationDataTable.StationInfoID WHERE CruiseTable.CruiseName LIKE 's%';"
cursor.execute(sql_stationData)
stationData = cursor.fetchall()
stationData_df = pd.DataFrame(stationData, columns=[i[0] for i in cursor.description])
print("Discrete Dataframe # rows matches # rows in StationDataTable:", len(discrete_df)==len(stationData_df))


No UW Duplicate columns by Values: True
No UW Duplicate columns by Name: True
Underway Dataframe # rows matches # rows in UWDataTable: True
No Discrete Duplicate columns by Values: True
No Discrete Duplicate columns by Name: True
Discrete Dataframe # rows matches # rows in StationDataTable: True


### Save data to CSVs:

In the database, null values were stored as None, Nan, and -999. Let's set all of these to -999.

In [37]:
uw_df.fillna(-999.0, inplace=True)
discrete_df.fillna(-999.0, inplace=True)
xbt_df.fillna(-999.0, inplace=True)

In [38]:
uw_df.to_csv('L:/mitchell/projects/nasacms2018/analysis/data/gnatsat_workflow/01-underway-gnats.csv', index=False)
discrete_df.to_csv('L:/mitchell/projects/nasacms2018/analysis/data/gnatsat_workflow/02-discrete-gnats.csv', index=False)
xbt_df.to_csv('L:/mitchell/projects/nasacms2018/analysis/data/gnatsat_workflow/03-xbt-gnats.csv', index=False)